# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
# from catboost import CatBoostClassifier

from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

In [5]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [6]:
class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : dict
        Словарь, отображающий метки в вероятность того, что объект, попавший в данный лист, принадлжит классу,
         соответствующиему метке.
    """
    def __init__(self, y_arr, n_classes):
        values, counts = np.unique(y_arr, return_counts=True)
        counts = counts / len(y_arr)
        self.dist = np.zeros(n_classes)
        self.dist[values] = counts
        # vc_zip = list(zip(values, counts))
        # self.preds = dict(vc_zip)
        # self.y = max(vc_zip, key=lambda x: x[1])[0]


class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value.
    """
    def __init__(self, split_dim: int, split_value: float,
                 left: Union['DecisionTreeNode', DecisionTreeLeaf],
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right


class DecisionTreeClassifier:
    """
    Attributes
    ----------
    root : Union[DecisionTreeNode, DecisionTreeLeaf]
        Корень дерева.

    (можете добавлять в класс другие аттрибуты).

    """

    def __init__(self, criterion: str = "gini",
                 max_depth: Optional[int] = None,
                 min_samples_leaf: int = 1.,
                 max_features: str = "auto"):
        """
        Parameters
        ----------
        criterion : str
            Задает критерий, который будет использоваться при построении дерева.
            Возможные значения: "gini", "entropy".
        max_depth : Optional[int]
            Ограничение глубины дерева. Если None - глубина не ограничена.
        min_samples_leaf : int
            Минимальное количество элементов в каждом листе дерева.

        """
        self.root = None
        criteria = {
            'gini': gini,
            'entropy': entropy
        }
        self.criterion = criteria[criterion]
        self.max_depth = max_depth if max_depth is not None else 1_000_000
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_classes = None
        self.oob_ids_mask = None

    def find_best_split(self, X: np.ndarray, y: np.ndarray, allowed_features: np.ndarray):
        n, m = X.shape
        split_gain, split_value, split_dim = -100, -1, -1
        for col in allowed_features:
            vals = np.unique(X[:, col])

            for val in vals:
                mask = X[:, col] < val
                left = y[mask]
                right = y[~mask]
                if len(left) < self.min_samples_leaf or len(right) < self.min_samples_leaf:
                    continue
                g = gain(left, right, self.criterion)
                if g > split_gain:
                    split_value = val
                    split_dim = col
                    split_gain = g

        return split_dim, split_value

    def build_tree(self, X: np.ndarray, y: np.ndarray, depth):
        if depth >= self.max_depth:
            return DecisionTreeLeaf(y, self.n_classes)

        features = np.random.choice(X.shape[1], self.max_features, replace=False)
        split_dim, split_value = self.find_best_split(X, y, features)

        if split_dim == -1:
            return DecisionTreeLeaf(y, self.n_classes)

        mask = X[:, split_dim] < split_value

        left = self.build_tree(X[mask], y[mask], depth + 1)
        right = self.build_tree(X[~mask], y[~mask], depth + 1)

        node = DecisionTreeNode(
            split_dim=split_dim,
            split_value=split_value,
            left=left,
            right=right
        )

        return node

    def fit(self, X: np.ndarray, y: np.ndarray) -> NoReturn:
        """
        Строит дерево решений по обучающей выборке.

        Parameters
        ----------
        X : np.ndarray
            Обучающая выборка.
        y : np.ndarray
            Вектор меток классов.
        """
        self.n_classes = np.unique(y).shape[0]
        self.max_features = int(np.ceil(np.sqrt(X.shape[1]))) if self.max_features == 'auto' else self.max_features
        self.root = self.build_tree(X, y, 0)

    def walk_down(self, node: Union[DecisionTreeNode, DecisionTreeLeaf], x):
        if type(node) is DecisionTreeLeaf:
            return node.dist
        split_value = node.split_value
        split_dim = node.split_dim
        if x[split_dim] < split_value:
            return self.walk_down(node.left, x)
        else:
            return self.walk_down(node.right, x)

    def predict_proba(self, X: np.ndarray) -> List[Dict[Any, float]]:
        """
        Предсказывает вероятность классов для элементов из X.

        Parameters
        ----------
        X : np.ndarray
            Элементы для предсказания.

        Return
        ------
        List[Dict[Any, float]]
            Для каждого элемента из X возвращает словарь
            {метка класса -> вероятность класса}.
        """
        preds = [self.walk_down(self.root, x) for x in X]
        return preds

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Предсказывает классы для элементов X.

        Parameters
        ----------
        X : np.ndarray
            Элементы для предсказания.

        Return
        ------
        list
            Вектор предсказанных меток для элементов X.
        """
        proba = self.predict_proba(X)
        proba = np.argmax(proba, axis=1)
        return proba

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [7]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators

        self.trees = [DecisionTreeClassifier(
            criterion=self.criterion,
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            max_features=self.max_features
        ) for _ in range(self.n_estimators)]
        self.n_classes = None
        self.label_encoder = LabelEncoder()
        self.out_bag_inds = np.ndarray

    def fit(self, X, y):
        n, m = X.shape

        self.n_classes = np.unique(y).shape[0]
        y = self.label_encoder.fit_transform(y)

        for i in range(len(self.trees)):
            out_bag_inds = np.zeros(n, dtype='bool')
            ids = np.random.randint(n, size=n)
            ids_uniq = np.unique(ids)
            out_bag_inds[ids_uniq] = True
            self.trees[i].oob_ids_mask = ~out_bag_inds
            self.trees[i].fit(X[ids], y[ids])

    def predict(self, X):
        preds = np.zeros((X.shape[0], self.n_classes))
        for tree in self.trees:
            preds += tree.predict_proba(X)

        preds = np.argmax(preds, axis=1)
        preds = self.label_encoder.inverse_transform(preds)
        return preds


### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [8]:
def feature_importance(rfc: RandomForestClassifier, X: np.ndarray, y: np.ndarray):
    n, m = X.shape
    importance = np.zeros((rfc.n_estimators, m))
    for i in range(len(rfc.trees)):
        inds = rfc.trees[i].oob_ids_mask
        X_, y_ = X[inds], y[inds]
        err_oob = np.mean(rfc.predict(X_) != y_)
        for col in range(m):
            X_shuffled = X_.copy()
            np.random.shuffle(X_shuffled[:, col])
            err_oob_col = np.mean(rfc.predict(X_shuffled) != y_)
            importance[i, col] = err_oob_col - err_oob

    importance = np.mean(importance, axis=0)
    return importance

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [12]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc, X, y))

Accuracy: 1.0
Importance: [0.         0.         0.19167063 0.19602083 0.44358922 0.        ]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [13]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [14]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [16]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc, X_train, y_age_train), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.6986128625472888
Most important features:
1. mudakoff
2. 4ch
3. rhymes
4. rapnewrap
5. dayvinchik
6. ovsyanochan
7. styd.pozor
8. iwantyou
9. bot_maxim
10. pravdashowtop
11. pixel_stickers
12. tumblr_vacuum
13. fuck_humor
14. vinevinevine
15. exclusive_muzic
16. leprum
17. 40kg
18. tnt
19. i_d_t
20. reflexia_our_feelings


#### Пол

In [17]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc, X_train, y_sex_train), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8360655737704918
Most important features:
1. 40kg
2. zerofat
3. mudakoff
4. girlmeme
5. rapnewrap
6. modnailru
7. 9o_6o_9o
8. cook_good
9. be.beauty
10. be.women
11. thesmolny
12. 4ch
13. femalemem
14. i_d_t
15. reflexia_our_feelings
16. woman.blog
17. beauty
18. igm
19. bon
20. sh.cook


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [ ]:
X, y = synthetic_dataset(1000)
print("Accuracy:", np.mean(None == y))
print("Importance:", None)

### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [ ]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [ ]:
print("Accuracy:", np.mean(None == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(None, features, 10)):
    print(str(i+1) + ".", name)

#### Пол

In [ ]:
print("Accuracy:", np.mean(None == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(None, features, 10)):
    print(str(i+1) + ".", name)